# Ventagium

## Task: Desarrollar un pipeline en Python que deberá:

1. Extraer los datos correspondientes al total de población de cada país usando la API JSON
de Indicadores del Banco Mundial.

- a. Debe recibir un parámetro para limitar la respuesta a la cantidad de países que se
soliciten. De tal forma que se puedan solicitar, por ejemplo, los 10, 20, 30 países con la
mayor población.

- b. Almacenar los datos extraídos en un objeto DataFrame de la librería Pandas.

In [14]:
import requests
import pandas as pd


# How to structure API/url
# https://datahelpdesk.worldbank.org/knowledgebase/articles/898581
# Population, total (SP.POP.TOTL)

# How to structure API Call:
#### https://datahelpdesk.worldbank.org/knowledgebase/articles/898581

Responses
Retrieving indicator data about countries is one common use of the API.

For example, the following is a call for 2006 data on the GDP of Brazil: https://api.worldbank.org/v2/country/br/indicator/NY.GDP.MKTP.CD?date=2006

Response Format
By default, all requests will respond with valid XML. To receive the response in JSON format, provide format=json in any request.


## using 'all' for all countries' data:

How to use the V2 Indicators API
Version 2 (V2) of the Indicators API has been released and replaces V1 of the API. V1 API calls will no longer be supported. To use the V2 API, you must place v2 in the call. For example: https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL.


In [18]:
pop_url = "https://api.worldbank.org/v2//country/all/indicator/SP.POP.TOTL?date=2020:2025&format=json"
response = requests.get(pop_url)
data = response.json()

In [23]:
records = data[1]

In [27]:

population_dict = {}

for item in records :
    country = item['country']['value']
    country_code = item['countryiso3code']
    year = int(item['date'])
    population_val = int(item['value'])

    if country not in population_dict:
        population_dict[country_code] = {'Country': country, 'Country Code': country_code}

    population_dict[country_code][f'pop_{year}'] = population_val

df = pd.DataFrame(population_dict.values())

df.tail(100)


,Country,Country Code,pop_2020,pop_2022
0,Africa Eastern and Southern,AFE,6.944461e+08,NaN
1,Africa Western and Central,AFW,4.745694e+08,NaN
2,Arab World,ARB,4.537232e+08,NaN
3,Caribbean small states,CSS,3.092577e+06,NaN
4,Central Europe and the Baltics,CEB,1.020466e+08,NaN
5,Early-demographic dividend,EAR,3.396051e+09,NaN
6,East Asia & Pacific,EAS,2.369439e+09,NaN
7,East Asia & Pacific (excluding high income),EAP,2.121866e+09,NaN
8,East Asia & Pacific (IDA & IBRD countries),TEA,2.095759e+09,NaN
9,Euro area,EMU,3.469224e+08,NaN
